In [1]:
import csv
import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

Using TensorFlow backend.


TensorFlow version:  2.1.0


In [2]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import average_precision_score

In [4]:
# Define paths
base_path = os.path.abspath("./")
dataset_path = os.path.join(base_path, "dataset", "isic2016numpy")
model_path = os.path.join(base_path, "models")
print(os.listdir(dataset_path))

['x_test.npy', 'x_testFeatures.npy', 'x_train.npy', 'x_trainFeatures.npy', 'y_test.npy', 'y_train.npy']


In [5]:
# Load data
x_train = np.load("{}/x_trainFeatures.npy".format(dataset_path)) 
y_train = np.load("{}/y_train.npy".format(dataset_path))
x_test = np.load("{}/x_testFeatures.npy".format(dataset_path))
y_test = np.load("{}/y_test.npy".format(dataset_path))

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
def construct_network(dropout, neuronPct, neuronShrink):
    neuronCount = int(neuronPct * 5000)
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Flatten(input_shape=x_train.shape[1:]))
    #model.add(Input(shape=(1,512)))
    #model.add(Flatten())
    model.add(Dense(neuronCount))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation='sigmoid')) # Output
    return model

In [8]:
x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255


In [9]:
class_weights = [0.1520195864872159, 0.8479804135127841]

In [10]:
class_weights 

[0.1520195864872159, 0.8479804135127841]

In [11]:
# Train the awesome model
# Configuration
batch_size = 2
epochs = 10

In [12]:
def optimize_network(dropout,lr,neuronPct,neuronShrink,alpha,gamma):
    output_dir = os.path.join(base_path, "weights")
    output_weights_name='weights.h5'
    start_time = time.time()
    model = construct_network(dropout, neuronPct, neuronShrink)
    optimizer = SGD(lr=lr)
    model.compile(optimizer=optimizer,loss=focal_loss(gamma=gamma,alpha=alpha))
    model.summary()
    EXP_NAME = str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)
    output_path = os.path.join(output_dir,EXP_NAME+"_"+output_weights_name)

    
    checkpoint = ModelCheckpoint(
                 output_path,
                 monitor='val_loss',
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
    )
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-8, mode='auto') # new_lr = lr * factor
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, patience=8, mode='auto', restore_best_weights=True)
    csv_logger = CSVLogger('{}\{}_training.csv'.format(output_dir, EXP_NAME))
    
    model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                class_weight = class_weights,
                callbacks=[csv_logger, early_stopping, reduce_lr, checkpoint], # early_stopping, checkpointer, reduce_lr
                shuffle=False)
    
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_flat = np.argmax(y_test, axis=1)
    
    y_pred_flat = np.array([np.argmax(pred) for pred in y_pred])
    
    acc = accuracy_score(y_test_flat, y_pred_flat) * 100
    print("Accuracy :", acc)
    
    auroc = roc_auc_score(y_test, y_pred) *100
    print("Auroc :", auroc)
    
    ap = average_precision_score(y_test, y_pred) * 100
    print("Average precision :", ap)
    
    
    cm = confusion_matrix(y_pred=y_pred_flat, y_true=y_test_flat)
    total=sum(sum(cm))

    sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
    print('Sensitivity : ', sensitivity*100 )

    Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
    print('Specificity : ', Specificity*100)
    
    if str(sensitivity) == "nan":
        sensitivity = 0
    
    return sensitivity 

In [13]:
from new_bayes_opt.bayesian_optimization import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

# Bounded region of parameter space
pbounds = { 'gamma':(0.1, 2.0),
            'alpha':(0.1, 2.0),
            'dropout': (0.05, 0.2),
           'lr': (0.009, 0.015),
           'neuronPct': (0.1850 , 2.000),
           'neuronShrink': (0.200, 0.400)
          }

#print(bounds.values())

In [14]:
optimizer = BayesianOptimization(
    f=optimize_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

Kunci ['alpha', 'dropout', 'gamma', 'lr', 'neuronPct', 'neuronShrink']
Bound  [[0.1   2.   ]
 [0.05  0.2  ]
 [0.1   2.   ]
 [0.009 0.015]
 [0.185 2.   ]
 [0.2   0.4  ]]


In [15]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
logger = JSONLogger(path="./1.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [16]:
start_time = time.time()
optimizer.maximize(init_points=5,acq="weightedei", n_iter=20, omega=0.9)
time_took = time.time() - start_time

Isine x [0.89234181 0.15804867 0.10021731 0.010814   0.45136194 0.21846772]
Konten (0.8923418089348906, 0.15804867401632372, 0.10021731215295529, 0.010813995435791039, 0.45136194183306017, 0.21846771895375958)
Masuk
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 492)               252396    
_________________________________________________________________
activation (Activation)      (None, 492)               0         
_________________________________________________________________
dropout (Dropout)            (None, 492)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 986       
Total params: 253,382
Trainable params

Epoch 5/10
846/900 [===========================>..] - ETA: 0s - loss: 0.4861
Epoch 00005: val_loss improved from 0.50527 to 0.50016, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10183410905645718_0.012232900404020142__0.94583804364198_0.3370439000793519_weights.h5
900/900 [==============================] - 1s 958us/sample - loss: 0.4823 - val_loss: 0.5002
Epoch 6/10
890/900 [============================>.] - ETA: 0s - loss: 0.4771
Epoch 00006: val_loss improved from 0.50016 to 0.49636, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10183410905645718_0.012232900404020142__0.94583804364198_0.3370439000793519_weights.h5
900/900 [==============================] - 1s 937us/sample - loss: 0.4767 - val_loss: 0.4964
Epoch 7/10
844/900 [===========================>..] - ETA: 0s - loss: 0.4763
Epoch 00007: val_loss improved from 0.49636 to 0.49358, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10183410905645718_0.01223290040402

876/900 [============================>.] - ETA: 0s - loss: 0.3119
Epoch 00001: val_loss improved from inf to 0.30559, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.07971522336273182_0.014809569454316385__0.7538648833590258_0.3384645231338628_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.3106 - val_loss: 0.3056
Epoch 2/10
884/900 [============================>.] - ETA: 0s - loss: 0.2931
Epoch 00002: val_loss improved from 0.30559 to 0.29871, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.07971522336273182_0.014809569454316385__0.7538648833590258_0.3384645231338628_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.2928 - val_loss: 0.2987
Epoch 3/10
898/900 [============================>.] - ETA: 0s - loss: 0.2852
Epoch 00003: val_loss improved from 0.29871 to 0.29390, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.07971522336273182_0.014809569454316385__0.7538648

Train on 900 samples, validate on 379 samples
Epoch 1/10
834/900 [==========================>...] - ETA: 0s - loss: 0.5357
Epoch 00001: val_loss improved from inf to 0.51989, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.11528410678765617_0.014861554283798768__0.9773507517808921_0.3194107131397793_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.5293 - val_loss: 0.5199
Epoch 2/10
856/900 [===========================>..] - ETA: 0s - loss: 0.5011
Epoch 00002: val_loss improved from 0.51989 to 0.50753, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.11528410678765617_0.014861554283798768__0.9773507517808921_0.3194107131397793_weights.h5
900/900 [==============================] - 1s 885us/sample - loss: 0.4987 - val_loss: 0.5075
Epoch 3/10
890/900 [============================>.] - ETA: 0s - loss: 0.4863
Epoch 00003: val_loss improved from 0.50753 to 0.49836, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\

Epoch 00010: early stopping
Accuracy : 79.41952506596306
Auroc : 50.757675438596486
Average precision : 50.24201589415419
Sensitivity :  80.70652173913044
Specificity :  36.36363636363637
Konten (1.2356505188691322, 0.09776192409755544, 1.359811641574782, 0.011690339137973502, 0.4975524474731156, 0.26058687130274083)
Konten (1.2356505188691322, 0.09776192409755544, 1.359811641574782, 0.011690339137973502, 0.4975524474731156, 0.26058687130274083)
Isine x [0.88921682 0.12535189 0.1826215  0.01368233 0.47032202 0.27940374]
Konten (0.889216816574261, 0.12535189210791717, 0.1826215029232542, 0.013682334729661221, 0.4703220189857832, 0.2794037434005523)
Masuk
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 656)            

...

846/900 [===========================>..] - ETA: 0s - loss: 0.4671
Epoch 00005: val_loss improved from 0.48357 to 0.47825, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10009510586494294_0.013790188897084575__0.5257209180780598_0.2328590836652262_weights.h5
900/900 [==============================] - 1s 973us/sample - loss: 0.4651 - val_loss: 0.4782
Epoch 6/10
886/900 [============================>.] - ETA: 0s - loss: 0.4592
Epoch 00006: val_loss improved from 0.47825 to 0.47376, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10009510586494294_0.013790188897084575__0.5257209180780598_0.2328590836652262_weights.h5
900/900 [==============================] - 1s 947us/sample - loss: 0.4580 - val_loss: 0.4738
Epoch 7/10
886/900 [============================>.] - ETA: 0s - loss: 0.4545
Epoch 00007: val_loss improved from 0.47376 to 0.46987, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10009510586494294_0.013790188897084575__0

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_20 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 1120)              574560    
_________________________________________________________________
activation_20 (Activation)   (None, 1120)              0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 1120)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 2)                 2242      
Total params: 576,802
Trainable params: 576,802
Non-trainable params: 0
_________________________________________________________________
Train on 900 samples, validate on 379 samples
Epoch 1/10
876/900 [============================>.] - ETA: 0s - los

868/900 [===========================>..] - ETA: 0s - loss: -21.6886
Epoch 00007: val_loss did not improve from -21.66000
900/900 [==============================] - 1s 1ms/sample - loss: -21.6886 - val_loss: -21.6600
Epoch 8/10
838/900 [==========================>...] - ETA: 0s - loss: -21.6886
Epoch 00008: val_loss did not improve from -21.66000
900/900 [==============================] - 1s 989us/sample - loss: -21.6886 - val_loss: -21.6600
Epoch 9/10
864/900 [===========================>..] - ETA: 0s - loss: -21.6886Restoring model weights from the end of the best epoch.

Epoch 00009: val_loss did not improve from -21.66000
900/900 [==============================] - 1s 995us/sample - loss: -21.6886 - val_loss: -21.6600
Epoch 00009: early stopping
Accuracy : 79.68337730870712
Auroc : 50.08662280701755
Average precision : 50.02751816877286
Sensitivity :  80.26666666666667
Specificity :  25.0
Konten (1.6802184692125663, 0.05110435673410891, 0.17444525075696907, 0.013846937798207334, 0.34

Epoch 2/10
838/900 [==========================>...] - ETA: 0s - loss: 0.5373
Epoch 00002: val_loss improved from 0.54865 to 0.54098, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1079319767327914_0.010596958363753295__0.4652010828041874_0.234365085544851_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.5334 - val_loss: 0.5410
Epoch 3/10
872/900 [============================>.] - ETA: 0s - loss: 0.5260
Epoch 00003: val_loss improved from 0.54098 to 0.53493, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1079319767327914_0.010596958363753295__0.4652010828041874_0.234365085544851_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.5262 - val_loss: 0.5349
Epoch 4/10
886/900 [============================>.] - ETA: 0s - loss: 0.5210
Epoch 00004: val_loss improved from 0.53493 to 0.52919, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1079319767327914_0.010596958363753295_

846/900 [===========================>..] - ETA: 0s - loss: 0.4855
Epoch 00009: val_loss improved from 0.50335 to 0.50047, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1591658033419711_0.009769250216129233__0.47011110520123955_0.23415698891490533_weights.h5
900/900 [==============================] - 1s 986us/sample - loss: 0.4826 - val_loss: 0.5005
Epoch 10/10
834/900 [==========================>...] - ETA: 0s - loss: 0.4859
Epoch 00010: val_loss improved from 0.50047 to 0.49743, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1591658033419711_0.009769250216129233__0.47011110520123955_0.23415698891490533_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4802 - val_loss: 0.4974
Accuracy : 80.21108179419525
Auroc : 71.42324561403508
Average precision : 66.62067075921519
Sensitivity :  80.21108179419525
Specificity :  nan
Konten (0.8704287335953145, 0.1591658033419711, 0.10940441642618083, 0.009769250216129233, 0.4701

In [17]:
def convert_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [18]:
print(f"Total runtime: {convert_string(time_took)}")
print(optimizer.max)

Total runtime: 0:03:52.19
{'target': 0.8128342245989305, 'params': {'alpha': 0.4460255468503901, 'dropout': 0.11528410678765617, 'gamma': 0.8814547502853013, 'lr': 0.014861554283798768, 'neuronPct': 0.9773507517808921, 'neuronShrink': 0.3194107131397793}}


In [19]:
import gc
gc.collect()

105038

In [20]:
new_params = {} 
for t in optimizer.max.keys():
    print(t)
    new_params = optimizer.max.get(t)    

target
params


In [21]:
a = new_params.get('alpha')
d = new_params.get('dropout')
g = new_params.get('gamma')
l = new_params.get('lr')
np = new_params.get('neuronPct')
ns = new_params.get('neuronShrink')
new_params.update({'alpha':(float(a-0.025),float(a+0.025)),
                   'dropout':(float(d-0.025),float(d+0.025)),
                   'gamma':(float(g-0.025),float(g+0.025)),
                   'lr':(float(l),float(l+0.025)),                   
                   'neuronPct':(float(np-0.025),float(np+0.025)),
                   'neuronShrink':(float(ns-0.025),float(ns+0.025))
                  }
                 )

In [22]:
new_params

{'alpha': (0.42102554685039006, 0.4710255468503901),
 'dropout': (0.09028410678765617, 0.14028410678765618),
 'gamma': (0.8564547502853013, 0.9064547502853013),
 'lr': (0.014861554283798768, 0.03986155428379877),
 'neuronPct': (0.9523507517808921, 1.002350751780892),
 'neuronShrink': (0.2944107131397793, 0.34441071313977933)}

In [23]:
w = csv.writer(open("new_params.csv", "w",newline=''))
for key, val in new_params.items():
    w.writerow([key, val])